In [5]:
import numpy as np
from icecream import ic

In [3]:
def compute_transformation_matrix(src_points, dst_points):
    assert len(src_points) == len(dst_points) == 3, "Need three points for a unique solution"
    
    A = []
    B = []
    
    for (src, dst) in zip(src_points, dst_points):
        A.append([src[0], src[1], 1, 0, 0, 0])
        A.append([0, 0, 0, src[0], src[1], 1])
        B.append(dst[0])
        B.append(dst[1])
    
    A = np.array(A)
    B = np.array(B)
    
    # Solve the linear system
    transform_params = np.linalg.solve(A, B)
    
    # Create the transformation matrix
    transformation_matrix = np.array([
        [transform_params[0], transform_params[1], transform_params[2]],
        [transform_params[3], transform_params[4], transform_params[5]],
        [0, 0, 1]
    ])
    
    return transformation_matrix

In [6]:
def transform_point(point, transformation_matrix):
    homogenous_point = np.array([point[0], point[1], 1])
    transformed_point = np.dot(transformation_matrix, homogenous_point)
    return transformed_point[:2]

In [ ]:
# Example points
#src_points = [(0, 0), (1, 0), (0, 1)]
#dst_points = [(2, 3), (3, 3), (2, 4)]

src_points = [(-0.242, -2.690),(-2.179, -3.207),(-0.773, -0.784)]
dst_points = [(0, 0),(2, 0),(0, 2)]


# Compute the transformation matrix
transformation_matrix = ic(compute_transformation_matrix(src_points, dst_points))


# Transform a point (x, y)
point1 = (-2.179, -3.207)
point2 = (-0.242, -2.690)
point3 = (-0.773, -0.784)
point4 = (-1.0,-1.0)

for point in [point1, point2, point3, point4]:
    transformed_point = ic(transform_point(point, transformation_matrix))

#print(f"Transformed Point: {transformed_point}")

In [ ]:
import matplotlib.pyplot as plt


# Separate the points into x and y coordinates for plotting
src_x, src_y = zip(*src_points)
dst_x, dst_y = zip(*dst_points)

# Create the plot
plt.figure(figsize=(8, 6))
plt.scatter(src_x, src_y, color='blue', label='Source Points')
plt.scatter(dst_x, dst_y, color='red', label='Destination Points')

# Add lines connecting corresponding points
for i in range(len(src_points)):
    plt.plot([src_points[i][0], dst_points[i][0]], [src_points[i][1], dst_points[i][1]], 'gray', linestyle='--')

# Add labels and title
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Source and Destination Points')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
